In [1]:
import csv 
import json 
from collections import Counter
#import spacy
#nlp = spacy.load('en_core_web_sm', disable=['ner'])
from tqdm import tqdm
import pandas as pd
from spacy.matcher import Matcher
from spacy.tokens import Span 
import nltk
from nltk.corpus import stopwords
skip_words = set(stopwords.words('english'))
import stanza
nlp = stanza.Pipeline('en')

from nltk.stem.wordnet import WordNetLemmatizer
lmtzr = WordNetLemmatizer()
from scripts.time_normalize import TimeNormalize

from sklearn import metrics 
from sklearn.linear_model import LogisticRegression
from sklearn import svm


do_lemmatize = True

2020-04-26 20:56:30 INFO: Loading these models for language: en (English):
| Processor | Package   |
-------------------------
| tokenize  | ewt       |
| pos       | ewt       |
| lemma     | ewt       |
| depparse  | ewt       |
| ner       | ontonotes |

2020-04-26 20:56:30 INFO: Use device: gpu
2020-04-26 20:56:30 INFO: Loading: tokenize
2020-04-26 20:56:32 INFO: Loading: pos
2020-04-26 20:56:33 INFO: Loading: lemma
2020-04-26 20:56:33 INFO: Loading: depparse
2020-04-26 20:56:34 INFO: Loading: ner
2020-04-26 20:56:35 INFO: Done loading processors!


ModuleNotFoundError: No module named 'scripts'

In [2]:
import unicodedata
def normalize(text):
    text = unicodedata.normalize('NFD', text).encode('ascii', 'ignore').decode("utf-8")
    return str(text)

In [3]:
def read_data(file):
    data = []
    with open(file, 'r') as f:
        for line in f:
            sample = line.strip().split('\t')
            assert len(sample) == 5
            data.append(sample)
    return data

In [4]:

def read_garbage(file, lead='verb'):
    garbage = {}
    concept_len = Counter()
    with open(file, 'r') as f:
        reader = csv.reader(f, delimiter=',')
        for i, line in enumerate(reader):
            if do_lemmatize:
                line[0] = lmtzr.lemmatize(line[0].lower())
                line[1] = lmtzr.lemmatize(line[1].lower())
                line[2] = lmtzr.lemmatize(line[2].lower())
            
            subj, verb, obj = line[0],line[1], line[2]
            if lead == 'verb':
                head1, head2, head3 = verb, subj, obj
            elif lead == 'subj': 
                head1, head2, head3 = subj, obj, verb
            else:
                head1, head2, head3 = obj, subj, verb
            if head1 not in garbage:
                garbage[head1] = {head2:{head3:[line[3]]}}
            elif head2 not in garbage[head1]:
                garbage[head1][head2] = {head3:[line[3]]}
            elif head3 not in garbage[head1][head2]:
                garbage[head1][head2][head3] = [line[3]]
            else:
                garbage[head1][head2][head3] += [line[3]]
        print (i)
    print (len(garbage))
    return garbage
# garbage_verb = read_garbage('/home/kaixinm/kaixinm/Wikipedia/python-sutime/DURATION.csv')
# garbage_subj = read_garbage('/home/kaixinm/kaixinm/Wikipedia/python-sutime/DURATION.csv', lead='subj')
# garbage_obj = read_garbage('/home/kaixinm/kaixinm/Wikipedia/python-sutime/DURATION.csv', lead='obj')
garbage_verb = read_garbage('./DURATION.csv')
garbage_subj = read_garbage('./DURATION.csv', lead='subj')
garbage_obj = read_garbage('./DURATION.csv', lead='obj')

1744894
29252
1744894
127539
1744894
45662


In [6]:
print (garbage_verb['log'])

{'he': {'mile': ['24-hour'], 'best': ['49.17 seconds']}, 'these': {'': ['2007-2018']}, '': {'': ['two weeks', 'four weeks', 'days', 'days', 'four weeks', '440 minutes', 'weeks', 'hours', 'up to 15 minutes', 'six weeks', 'weeks', '45 hours'], 'hour': ['week', 'hours'], 'tree': ['day']}, 'site': {'visitor': ['a day']}, 'it': {'': ['week']}, 'truck': {'mile': ['a year']}, 'rider': {'': ['hours']}, 'census': {'growth': ['ten-year']}, 'she': {'goal': ['37 seconds']}, 'supervisor': {'minute': ['four hour'], '': ['thirty minutes']}, 'bridge': {'': ['days']}, 'lp': {'': ['week'], 'week': ['the past five years']}, 'lifespan': {'': ['seven weeks']}, 'which': {'': ['ten hours']}, 'water': {'': ['several days']}}


In [38]:
def check_role(label):
    print (label)
    if 'subj' in label or 'obj' in label:
        return True
    else:
        return False
def extract_info(content):
    doc = nlp(content)
    query = [[], [], []]
    all_words = doc.sentences[0].words
    root = [w for w in doc.sentences[0].words if w.deprel == 'root'][0]
    if content.startswith('it'):
        print (content)
        #print (all_words)
    if not root.xpos.startswith('VB'):
        #print (all_words)
        if root.xpos.startswith('JJ'):
            #query[1].append(root.text)
            for w in doc.sentences[0].words:
                if 'root' in w.deprel:
                    query[1].append(w.text)
                if 'subj' in w.deprel:
                    query[0].append(w.text)
                if 'nmod' in w.deprel:
                    query[2].append(w.text)
            #print (content)
        else:
        #elif root.xpos.startswith('NN'):
            #query[0].append(root.text)
            for w in doc.sentences[0].words:
                if 'root' in w.deprel:
                    query[0].append(w.text)
                if 'amod' in w.deprel and len(query[0]) != 0:
                    query[1].append(w.text)
                if 'nmod' in w.deprel and len(query[0]) != 0:
                    query[2].append(w.text)
            #print (content)
        #else:
        #    print (all_words)
        #if 'last' in content:
        #    query[1].append('last')
    else:
        for w in  doc.sentences[0].words:

            if w.head == 0:
                query[1].append(w.text)
            if (all_words[w.head-1].head == 0 and 'subj' in w.deprel):
                query[0].append(w.text)
            if (all_words[w.head-1].head == 0 and 'obj' in w.deprel):
                query[2].append(w.text)
    query = [' '.join(part) for part in query]
    #print (query)
    return query

from collections import defaultdict
def find_patterns(data):
    # return: dict 
    # key: question str 
    # value: tokenized_query: list, level: int, raw_times: list
    start = Counter()
    prev = ''
    question_to_range_map = defaultdict(lambda: defaultdict(dict))
    # question_to_range_map = defaultdict()
    for sample in data:
        if sample[-1] == 'Event Duration':
            if prev == sample[1]:
                continue
            prev = sample[1]
            question = nltk.word_tokenize(sample[1].lower())
            q_tags = nltk.pos_tag(question)
            if do_lemmatize:
                question = [lmtzr.lemmatize(word) for word in question]

            if 'how' not in question:
                content = sample[1].lower()
            # hardcode unhandled case for now
            elif sample[1] == 'How long will the Earth act like a magnet?':
                content = 'earth act like a magnet ?'
            elif not q_tags[2][1].startswith('VB') and 'would' not in question:
                for i in range(3, len(q_tags)):
                    if q_tags[i][1].startswith('VB'):
                        break
                if len(question)>i+3 and question[i+1] == 'it' and (question[i+2] == 'take' or question[i+2] == 'taken'):
                    q_words = ' '.join(question[:i+3])
                    content = ' '.join(question[i+3:])
                else:
                    q_words = ' '.join(question[:i+1])
                    content = ' '.join(question[i+1:])
            else:
                if len(question)>5 and question[3] == 'it' and (question[4] == 'take' or question[4] == 'taken'):
                    q_words = ' '.join(question[:5])
                    content = ' '.join(question[5:])
                else:
                    q_words = ' '.join(question[:3])
                    content = ' '.join(question[3:])
            start[q_words] += 1
            if len(content) == 0:
                continue
            # ['minority', 'remained', '']
            query = extract_info(content)
            level, results = search_in_garbage(query, garbage_verb)
            if level == 2:
                subj_level, subj_results = search_in_garbage_subj(query, garbage_subj, garbage_obj)
                if subj_level == 2:
                    results += subj_results
            elif level == 1:
                subj_level, subj_results = search_in_garbage_subj(query, garbage_subj, garbage_obj)
                if subj_level == 2:
                    results = subj_results 
                    level = subj_level
                elif subj_level == 1:
                    results += subj_results
            elif level == 0:
                level, results = search_in_garbage_subj(query, garbage_subj, garbage_obj)
#             print (query)
#             print (level)
#             print (results)
#             break
            
            question_to_range_map[sample[1]]['tokenized_query'] = query
            question_to_range_map[sample[1]]['level'] = level
            # question_to_range_map[sample[1]]['raw_times'] = results
            question_to_range_map[sample[1]]['raw_times'] = [normalize(i.lower().replace('-', ' ')) for i in results]
            # start converting results to range
    #print (start)
    return question_to_range_map

In [8]:
def search_in_garbage(query, garbage):
#     import pdb; pdb.set_trace()
    if query[1] != '' and query[1] in garbage:
        if query[0] != '' and query[0] in garbage[query[1]]:
            if query[2] != '' and query[2] in garbage[query[1]][query[0]]:
                results = garbage[query[1]][query[0]][query[2]]      # all 3 words matched
                return 3, results   # how many words matched
            elif query[2] != '':   # there exists object but not matched 
                results = [vv for obj, v in garbage[query[1]][query[0]].items() for vv in v]  # stuff matched by verb and subj
                for subj, d in garbage[query[1]].items():  # search for all that could be matched by verb and obj
                    for obj, v in d.items():
                        if obj == query[2]:
                            results += v
                return 2, results     # all level 2 matched
            else:
                results = [vv for obj, v in garbage[query[1]][query[0]].items() for vv in v]  # only has verb and subj, already the best we can do
                return 2, results 
        elif query[2] != '':  # subj not matched and obj exist
            results = []
            for subj, d in garbage[query[1]].items():  # search for all that could be matched by verb and obj
                for obj, v in d.items():
                    if obj == query[2]:
                        results += v
            if len(results) > 0:
                return 2, results
            else:        # the best we can do is matching only with verb
                for subj, d in garbage[query[1]].items():
                    for obj, v in d.items():
                        results += v
                return 1, results
        else:   # the best we can do is matching only with verb
            results = []
            for subj, d in garbage[query[1]].items():
                for obj, v in d.items():
                    results += v
            return 1, results
    return 0, []

In [9]:
def search_in_garbage_subj(query, garbage, next_garbage):
    if query[0] != '' and query[0] in garbage:
        if query[2] != '' and query[2] in garbage[query[0]]:  # we search for subj and obj match
                results = [vv for obj, v in garbage[query[0]][query[2]].items() for vv in v]
                return 2, results 
        elif query[2] != '':  # obj not matched but exist
            results = []   # all matched with subj
            for obj, d in garbage[query[0]].items():
                for verb, v in d.items():
                    results += v
            results += search_in_garbage_obj(query, next_garbage)
            return 1, results
        else:   # the best we can do is matching only with subj
            results = []
            for obj, d in garbage[query[0]].items():
                for verb, v in d.items():
                    results += v
            return 1, results
    elif query[2] != '':
        results = []
        results += search_in_garbage_obj(query, next_garbage)
        return 1, results
    return 0, []
def search_in_garbage_obj(query, garbage):
    if query[2] != '' and query[2] in garbage:
        results = []
        for subj, d in garbage[query[2]].items():
            for verb, v in d.items():
                results += v
        return results
    return []

In [10]:
def expand_map_with_range(question_to_range_map):
    unit_keys = ['seconds', 'minutes', 'hours', 'days', 'weeks', 'months', 'years', 'decades', 'centuries']
    if do_lemmatize:
        unit_keys = [lmtzr.lemmatize(key) for key in unit_keys]
    for k, v in question_to_range_map.items():
        unit_count_map = defaultdict(lambda:0, {k: 0 for k in unit_keys})
        # raw_times = [normalize(i.replace('-', ' ')) for i in v['raw_times']]
        raw_times = v['raw_times']
        for time in raw_times:
            time_tokenized = nltk.word_tokenize(time)
            if do_lemmatize:
                time_tokenized = [lmtzr.lemmatize(tok) for tok in time_tokenized]
            for tok in time_tokenized:
                if tok in unit_count_map:
                    unit_count_map[tok] += 1

        # sort unit_count_map desc
        if second_sort_by_desc_unit:
            unit_count_sorted = sorted(unit_count_map.items(), key=lambda v: (v[1], unit_keys.index(v[0])), reverse=True)
        else:
            unit_count_sorted = sorted(unit_count_map.items(), key=lambda v: v[1], reverse=True)
        question_to_range_map[k]['unit_count_sorted'] = unit_count_sorted
        question_to_range_map[k]['max_unit'] = unit_count_sorted[0]
        question_to_range_map[k]['min_unit'] = unit_count_sorted[1]
        # generate range
        if unit_keys.index(unit_count_sorted[0][0]) < \
            unit_keys.index(unit_count_sorted[1][0]):
            all_ranged_units = unit_keys[unit_keys.index(unit_count_sorted[0][0]):unit_keys.index(unit_count_sorted[1][0])+1]
        else:
            all_ranged_units = unit_keys[unit_keys.index(unit_count_sorted[1][0]):unit_keys.index(unit_count_sorted[0][0])+1]
            
        question_to_range_map[k]['range'] = all_ranged_units
    return question_to_range_map

In [11]:
def get_result_from_range(norm_data, range_map):
    results = []
    for sample in norm_data:
        if sample[-1] == 'Event Duration':
            ranged_units = range_map[sample[1]]['range']
            answer = nltk.word_tokenize(sample[2].lower())
            answer = [lmtzr.lemmatize(word) for word in answer]
            found = False
            for tok in answer:
                if tok in ranged_units:
                    found = True
                    break
            results.append('yes' if found else 'no')
    return results
            
    

In [69]:
do_lemmatize = True
second_sort_by_desc_unit = False
# data = read_data('../../../MCTACO/dataset/test_9442.tsv')
data = read_data('./MCTACO/dataset/dev_3783.tsv')
question_to_time_map = find_patterns(data)
question_to_range_map = expand_map_with_range(question_to_time_map)
normalized_data = read_data('./MCTACO/dataset/normalized_dev_3783.tsv')
res = get_result_from_range(normalized_data, question_to_range_map)

In [27]:
with open('./MCTACO/dataset/garbage_duration_result.txt', 'w') as f:
    for ans in res:
        f.write(ans+'\n')

In [33]:
import random
random.seed(15213)
def dump_baselines(res):
    with open('../../../MCTACO/dataset/random_result.txt', 'w') as f:
        for ans in res:
            if random.random() > 0.5:
                f.write('yes\n')
            else:
                f.write('no\n')
    with open('../../../MCTACO/dataset/positive_result.txt', 'w') as f:
        for ans in res:
            f.write('yes\n')
    with open('../../../MCTACO/dataset/negative_result.txt', 'w') as f:
        for ans in res:
            f.write('no\n')
dump_baselines(res)

In [ ]:
# Dev random
# Strict Acc.: 0.031746031746031744
# Avg F1: 0.3056352377780948
# Dev negative
# Strict Acc.: 0.24603174603174602
# Avg F1: 0.24603174603174602
# Dev positive
# Strict Acc.: 0.015873015873015872
# Avg F1: 0.3305273982573848
# Dev garbage 
# Strict Acc.: 0.19047619047619047
# Avg F1: 0.38031987389756794


In [ ]:
# Test positive 
# Strict Acc.: 0.022292993630573247
# Avg F1: 0.37325883850538083
# Test negative
# Strict Acc.: 0.2197452229299363
# Avg F1: 0.2197452229299363
# Test random 
# Strict Acc.: 0.028662420382165606
# Avg F1: 0.28177336796669894
# Test garbage 
# Strict Acc.: 0.15605095541401273
# Avg F1: 0.3671067949883198

In [50]:
# 1. lemmatize garbage subj/obj/verb
# 2. start generating ranges for each Q 

In [22]:
def write_data(filename, data):
    with open(filename, 'w') as fout:
        json.dump(data, fout)
write_data('dev_3783_garbage_part1.json', question_to_time_map)

## normalize all extracted patterns to seconds and
 1. use min max as pure range for rule based extraction
 2. use (min, max, avg, std) as feature for logistic regression

In [12]:
norm = TimeNormalize()

In [13]:
# convert question_to_range_map 
import statistics

def expand_map_with_seconds_stats(norm, question_to_range_map, add_quantization=False, drop_century=True):
    """expand dictionary and add fields 'converted_seconds', 'min', 'max', 'avg', 'std', 'quantization'(optional)
    norm(object)
    question_to_range_map(dict) 
    """
    if add_quantization:
        raise NotImplementedError
    for k, v in question_to_range_map.items():
        question_to_range_map[k]['converted_seconds'] = []
        for t in v['raw_times']:
            if drop_century and ('century' in t or 'centuries' in t):
                continue
            success, second = norm.convert_time_to_seconds(t)
            if success:
                question_to_range_map[k]['converted_seconds'].append(second)
        
        # get stats
        try:
            question_to_range_map[k]['min'] = min(question_to_range_map[k]['converted_seconds'])
            question_to_range_map[k]['max'] = max(question_to_range_map[k]['converted_seconds'])
            question_to_range_map[k]['avg'] = statistics.mean(question_to_range_map[k]['converted_seconds'])
            question_to_range_map[k]['std'] = statistics.stdev(question_to_range_map[k]['converted_seconds'])
        except ValueError:
            if len(question_to_range_map[k]['converted_seconds']) == 1:
                question_to_range_map[k]['std'] = 0.0
            elif len(question_to_range_map[k]['converted_seconds']) == 0:
                question_to_range_map[k]['min'] = 0.0
                question_to_range_map[k]['max'] = 0.0
                question_to_range_map[k]['avg'] = 0.0
                question_to_range_map[k]['std'] = 0.0
            else:
                print(k)
                print(question_to_range_map[k])
#         if add_quantization:
            # quantize by ['seconds', 'minutes', 'hours', 'days', 'weeks', 'months', 'years', 'decades', 'centuries']
            
            
    return question_to_range_map
    

In [14]:
def get_result_from_seconds_min_max(data, range_map):
    results = []
    for sample in data:
        if sample[-1] == 'Event Duration':
            min_second = range_map[sample[1]]['min']
            max_second = range_map[sample[1]]['max']
            answer = float(sample[2])
            if min_second <= answer <= max_second:
                results.append('yes')        
            else:
                results.append('no')
    return results

In [79]:
def write_eval_file(result, path):
    with open(path, 'w') as f:
        for ans in result:
            f.write(ans+'\n')

1. use min max as pure range for rule based extraction

In [71]:
question_to_range_map = expand_map_with_seconds_stats(norm, question_to_range_map, add_quantization=False, drop_century=True)

In [76]:
data = read_data('./MCTACO/dataset/seconds_dev_event_duration.tsv')
res = get_result_from_seconds_min_max(data, question_to_range_map)

In [80]:
path = './MCTACO/dataset/min_max_seconds.txt'
write_eval_file(res, path)

In [82]:
!python evaluator.py eval --test_file MCTACO/dataset/normalized_dev_event_duration.tsv --prediction_file MCTACO/dataset/min_max_seconds.txt

Strict Acc.: 0.09523809523809523
Avg F1: 0.3410240889250913


2. use (min, max, avg, std) as feature for logistic regression

In [16]:
# convert map to pandas
def convert_map_to_matrix(range_map, data, unit_fields, additional_feature_fields=['min', 'max', 'avg', 'std']):
    rows = []
    
    for sample in data:
        row = []
        if sample[-1] == 'Event Duration':
            exist_fields = [i[0] for i in range_map[sample[1]]['unit_count_sorted']]
            for f in unit_fields:
                if f in exist_fields:
                    # find its index from sorted list and get counts
                    index = exist_fields.index(f)
                    row.append(range_map[sample[1]]['unit_count_sorted'][index][1])
                else:
                    row.append(0)
    
            
            for field in additional_feature_fields:
                row.append(range_map[sample[1]][field])
            # append seconds of this instance
            row.append(float(sample[2]))
            # append label to the end
            if sample[-2] == 'yes':
                row.append(1)
            else:
                row.append(0)
            rows.append(row)
    return rows
        

In [43]:
do_lemmatize = True
second_sort_by_desc_unit = False
unit_fields = ['second', 'minute', 'hour', 'day', 'week', 'month', 'year', 'decade', 'century']
stats_fields = ['min', 'max', 'avg', 'std']
train_data = read_data('./MCTACO/dataset/seconds_dev_event_duration.tsv')
train_question_to_time_map = find_patterns(train_data)
train_question_to_range_map = expand_map_with_range(train_question_to_time_map)
train_question_to_range_map = expand_map_with_seconds_stats(norm, train_question_to_range_map, add_quantization=False, drop_century=True)
train_rows = convert_map_to_matrix(train_question_to_range_map, train_data, unit_fields, additional_feature_fields=stats_fields)




test_data = read_data('./MCTACO/dataset/seconds_test_event_duration.tsv')
test_question_to_time_map = find_patterns(test_data)
test_question_to_range_map = expand_map_with_range(test_question_to_time_map)
test_question_to_range_map = expand_map_with_seconds_stats(norm, test_question_to_range_map, add_quantization=False, drop_century=True)
test_rows = convert_map_to_matrix(test_question_to_range_map, test_data, unit_fields, additional_feature_fields=stats_fields)

it existed ?
it last ?
it a violent world


In [44]:
all_fields = unit_fields + stats_fields + ['candidate_seconds','label']
train_df = pd.DataFrame(train_rows,columns=all_fields)
test_df = pd.DataFrame(test_rows,columns=all_fields)

In [45]:
test_df.head(200)

,second,minute,hour,day,week,month,year,decade,century,min,max,avg,std,candidate_seconds,label
0,0,0,0,0,0,0,1,0,0,0.0,0.000000e+00,0.000000e+00,0.000000e+00,3.000000e+01,0
1,0,0,0,0,0,0,1,0,0,0.0,0.000000e+00,0.000000e+00,0.000000e+00,1.892160e+10,0
2,0,0,0,0,0,0,1,0,0,0.0,0.000000e+00,0.000000e+00,0.000000e+00,2.838240e+09,0
3,0,0,0,0,0,0,1,0,0,0.0,0.000000e+00,0.000000e+00,0.000000e+00,1.555200e+07,1
4,0,0,0,0,0,0,1,0,0,0.0,0.000000e+00,0.000000e+00,0.000000e+00,3.600000e+02,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,1,1,7,30,10,10,78,3,0,3.0,1.576800e+09,1.874131e+08,3.370173e+08,2.592000e+07,0
196,1,1,7,30,10,10,78,3,0,3.0,1.576800e+09,1.874131e+08,3.370173e+08,3.000000e+02,1
197,1,1,7,30,10,10,78,3,0,3.0,1.576800e+09,1.874131e+08,3.370173e+08,6.000000e+02,1
198,1,1,7,30,10,10,78,3,0,3.0,1.576800e+09,1.874131e+08,3.370173e+08,8.640000e+05,0


In [46]:
train_x = train_df.drop('label',axis = 1)
train_y = train_df.label
test_x = test_df.drop('label', axis=1)
test_y = test_df.label

In [78]:
logistic_regression = LogisticRegression()
# clf = svm.SVC()
# clf.fit(train_x, train_y)
# pred = clf.predict(test_x)

logistic_regression.fit(train_x, train_y)
pred_y = logistic_regression.predict(test_x)

/home/xiaopen2/miniconda3/envs/py36/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:939: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html.
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [77]:
metrics.accuracy_score(test_y, pred)

0.7404353562005277

In [81]:
len([i for i in pred_y if i == 1])

0

In [25]:
# for k, v in test_question_to_range_map.items():
# #     if len(v['min'])==0 or len(v['max'])==0 or len(v['avg'])==0 or len(v['std'])==0:
#     try:
#         a = float(v['min'])
#         a = float(v['max'])
#         a = float(v['avg'])
#         a = float(v['std'])
#     except:
#         print(k)
#         print(v)

How long will the Earth act like a magnet?
defaultdict(<class 'dict'>, {'unit_count_sorted': {}, 'min': {}, 'max': {}, 'avg': {}, 'std': {}})
